In [ ]:
#!pip install tensorflow
#!pip install keras-tuner

In [2]:
# importing the libraries
import keras
from keras import layers
from keras.layers import Dense, Flatten
from keras.models import Model
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras_tuner import RandomSearch

import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [3]:
# re-sizing all the images
IMAGE_SIZE = [224, 224]

train_path = '/content/drive/MyDrive/Dataset_cotton/Dataset/train'
valid_path = '/content/drive/MyDrive/Dataset_cotton/Dataset/val'

In [4]:
# Using  ImageDataGenerator to import the images from the dataset
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [5]:
#training and test dataset
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/Dataset_cotton/Dataset/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/Dataset_cotton/Dataset/val',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 1951 images belonging to 4 classes.
Found 324 images belonging to 4 classes.


In [7]:
labels = test_set.class_indices
print(labels)

{'diseased cotton leaf': 0, 'diseased cotton plant': 1, 'fresh cotton leaf': 2, 'fresh cotton plant': 3}


In [8]:
#Model Building

def build_model(hp):
  model = keras.Sequential()

  #finding best no of layers with parameters
  for i in range (hp.Int('no_conv_layers', min_value = 2, max_value =5)):
    model.add(layers.Conv2D(filters= hp.Int(f'filter{i}', min_value =32 , max_value = 128, step = 16),
                            kernel_size= hp.Choice(f'kernel{i}' , values = [2 , 4] ),
                            activation = 'relu',
                            input_shape = (224, 224 ,3)))

    model.add(layers.MaxPool2D(hp.Int(f'pool_size {i}' , min_value = 2 , max_value = 3 , step = 1)))


  model.add(layers.Flatten())


  #finding the best no of dense layers
  #fully connected layer

  for i in range(hp.Int('no_of_denseLayer' , min_value = 1, max_value=3)):
    model.add(layers.Dense(
        units = hp.Int(f'dense_unit{i}', min_value = 32 , max_value = 256, step= 32),
        activation = 'relu'
        ))

  #final output layer

  model.add(layers.Dense(4 , activation = 'softmax'))



  model.compile(optimizer= keras.optimizers.Adam(
                            hp.Choice('learning rate', values=[1e-2 , 1e-3, 1e-4])),
                loss = 'categorical_crossentropy',
                metrics = ['accuracy']     )

  return model





In [10]:
#instantiating Keras tunner RandomSearch
tuner = RandomSearch(
      build_model,
      objective = 'val_accuracy',
      max_trials=  7,
      directory = 'Dataset_cotton',
      project_name = 'Dataset'


)

In [11]:
tuner.search(training_set, epochs=7, validation_data=test_set)

Trial 7 Complete [00h 04m 43s]
val_accuracy: 0.8395061492919922

Best val_accuracy So Far: 0.8395061492919922
Total elapsed time: 00h 49m 15s


In [12]:
#best_parameter = tuner.get_best_hyperparameters()


In [13]:
# taking best model
best_model = tuner.get_best_models(num_models=1)[0]

In [14]:
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 221, 221, 64)      3136      
                                                                 
 max_pooling2d (MaxPooling2  (None, 110, 110, 64)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 107, 107, 64)      65600     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 35, 35, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 32, 32, 112)       114800    
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 16, 16, 112)       0

In [15]:
best_model.fit(training_set, epochs =45 , initial_epoch =7 , validation_data=test_set)

Epoch 8/45
61/61 [==============================] - 38s 592ms/step - loss: 0.5422 - accuracy: 0.7970 - val_loss: 0.5816 - val_accuracy: 0.7562
Epoch 9/45
61/61 [==============================] - 39s 637ms/step - loss: 0.4157 - accuracy: 0.8355 - val_loss: 0.3063 - val_accuracy: 0.8827
Epoch 10/45
61/61 [==============================] - 36s 591ms/step - loss: 0.3932 - accuracy: 0.8534 - val_loss: 0.3124 - val_accuracy: 0.8735
Epoch 11/45
61/61 [==============================] - 35s 580ms/step - loss: 0.3490 - accuracy: 0.8642 - val_loss: 0.2773 - val_accuracy: 0.8735
Epoch 12/45
61/61 [==============================] - 35s 572ms/step - loss: 0.2708 - accuracy: 0.8949 - val_loss: 0.2589 - val_accuracy: 0.8981
Epoch 13/45
61/61 [==============================] - 36s 583ms/step - loss: 0.2646 - accuracy: 0.8980 - val_loss: 0.3734 - val_accuracy: 0.8704
Epoch 14/45
61/61 [==============================] - 35s 566ms/step - loss: 0.2481 - accuracy: 0.9077 - val_loss: 0.4464 - val_accuracy: 0

In [16]:
y_pred = best_model.predict(test_set)

11/11 [==============================] - 2s 169ms/step


In [19]:
import numpy as np
y_pred = np.argmax(y_pred, axis=1)

In [22]:
y_pred

array([3, 1, 0, 0, 2, 0, 1, 1, 1, 1, 1, 3, 3, 3, 0, 2, 3, 1, 3, 1, 2, 1,
       1, 2, 2, 1, 1, 3, 0, 3, 2, 2, 1, 1, 1, 3, 3, 0, 0, 0, 1, 2, 3, 3,
       1, 1, 1, 0, 1, 3, 3, 2, 1, 1, 3, 2, 3, 2, 0, 0, 3, 3, 3, 0, 1, 3,
       3, 0, 0, 1, 1, 1, 1, 1, 3, 2, 2, 1, 1, 0, 3, 3, 2, 2, 1, 3, 3, 3,
       1, 3, 1, 1, 1, 3, 3, 0, 3, 1, 0, 3, 3, 3, 1, 1, 2, 3, 0, 0, 3, 3,
       2, 2, 1, 2, 2, 1, 0, 0, 3, 2, 3, 2, 1, 3, 2, 1, 1, 0, 3, 2, 3, 3,
       1, 2, 1, 0, 1, 0, 1, 2, 2, 1, 0, 3, 2, 3, 0, 1, 1, 3, 2, 1, 2, 3,
       1, 3, 1, 1, 1, 2, 2, 1, 1, 2, 1, 3, 2, 1, 1, 2, 0, 3, 2, 1, 0, 1,
       0, 1, 3, 0, 0, 1, 1, 3, 3, 3, 1, 1, 0, 3, 3, 1, 1, 1, 3, 1, 1, 3,
       1, 0, 0, 0, 0, 0, 3, 1, 2, 2, 2, 2, 1, 3, 1, 2, 2, 2, 3, 2, 0, 1,
       1, 1, 2, 3, 3, 1, 3, 2, 1, 1, 1, 3, 2, 3, 1, 0, 2, 2, 0, 1, 1, 1,
       1, 1, 2, 1, 1, 0, 1, 3, 2, 1, 2, 2, 1, 3, 1, 3, 0, 2, 3, 1, 2, 2,
       1, 1, 1, 3, 3, 1, 3, 2, 3, 2, 3, 3, 0, 0, 3, 1, 2, 3, 2, 3, 2, 1,
       0, 0, 0, 1, 0, 2, 2, 0, 2, 1, 1, 3, 1, 2, 0,

In [24]:
from keras.models import load_model

best_model.save("cnn_cotton.h5")

In [1]:
#PREDICTION

from keras.models import load_model
from keras.preprocessing import image
from keras.utils import load_img

In [3]:
model=load_model('cnn_cotton.h5')

In [6]:
img= load_img(r'Dataset\test\fresh cotton leaf\d (378).jpg',target_size=(224,224))

In [7]:
x=image.img_to_array(img)
x

array([[[ 50.,  43.,  35.],
        [ 53.,  48.,  42.],
        [ 63.,  56.,  50.],
        ...,
        [157., 190., 135.],
        [138., 174., 104.],
        [159., 193., 142.]],

       [[ 83.,  70.,  61.],
        [103.,  93.,  83.],
        [110.,  98.,  84.],
        ...,
        [121., 159.,  72.],
        [167., 197., 137.],
        [171., 204., 151.]],

       [[125., 111.,  98.],
        [152., 134., 120.],
        [166., 149., 133.],
        ...,
        [156., 187., 120.],
        [140., 175., 107.],
        [152., 188., 118.]],

       ...,

       [[182., 166., 151.],
        [179., 166., 149.],
        [172., 159., 143.],
        ...,
        [173., 202., 158.],
        [166., 193., 148.],
        [177., 203., 158.]],

       [[136., 124., 112.],
        [159., 146., 130.],
        [160., 148., 132.],
        ...,
        [183., 206., 160.],
        [177., 203., 158.],
        [176., 202., 155.]],

       [[104.,  95.,  80.],
        [124., 111.,  95.],
        [152., 1

In [8]:
x.shape

(224, 224, 3)

In [9]:
x = x/255

In [11]:
import numpy as np
x=np.expand_dims(x,axis=0)

x.shape

(1, 224, 224, 3)

In [12]:
model.predict(x)

1/1 [==============================] - 1s 561ms/step


array([[1.0361809e-05, 6.3583491e-07, 9.9998748e-01, 1.4974540e-06]],
      dtype=float32)

In [13]:
a=np.argmax(model.predict(x), axis=1)

1/1 [==============================] - 0s 92ms/step


In [14]:
a

array([2], dtype=int64)

In [ ]:
# Downloading the model to drive

#from google.colab import drive
#drive.mount('/content/drive')

#import shutil
#shutil.copy("/content/cnn_cotton.h5", "/content/drive/MyDrive/Dataset_cotton")